<a href="https://colab.research.google.com/github/ussnllmn/Deep_Learning/blob/main/Test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
''' IMPORT ALL THE THINGS '''
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
''' DEFINE PATH TO THE DATASETS '''

'''CWD = os.getcwd().replace('\\','/') # get current workspace directory
IMAGE_DIR = f'{CWD}/images'
TRAIN_DATASET_DIR = f'{IMAGE_DIR}/train'
TEST_DATASET_DIR = f'{IMAGE_DIR}/test'
VAL_DATASET_DIR = f'{IMAGE_DIR}/validation'

CATEGORIES = ['clementine','grapefruit','orange']'''

# print('CWD :',CWD)
# print('IMAGE_DIR :',IMAGE_DIR)
# print('TRAIN_DATASET_DIR :',TRAIN_DATASET_DIR)
# print('TEST_DATASET_DIR :',TEST_DATASET_DIR)
# print('VAL_DATASET_DIR :',VAL_DATASET_DIR)
# print('CATEGORIES :', CATEGORIES)

**กำหนด path ใหม่ เพราะ run จาก machine บน google colab**

In [ ]:
train = ImageDataGenerator(rescale= 1/255)
validation = ImageDataGenerator(rescale= 1/255)

In [ ]:
train_dataset = train.flow_from_directory("/content/drive/MyDrive/Images_of_Waste/train",
                                          target_size=(224,224),
                                          batch_size = 3,
                                          class_mode ="categorical")

validation_dataset = validation.flow_from_directory("/content/drive/MyDrive/Images_of_Waste/valid",
                                          target_size=(224,224),
                                          batch_size = 3,
                                          class_mode ="categorical")

In [ ]:
train_dataset.class_indices

In [ ]:
validation_dataset.class_indices

In [ ]:
''' Create Augmented Data '''

In [ ]:
''' Get pre train model '''
VGG = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
VGG.trainable = False

In [ ]:
''' Create a model '''
model = tf.keras.models.Sequential([
                                    VGG,
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.summary() # show all the model's layer

In [ ]:
''' Compiling '''

opt = tf.keras.optimizers.Adam(learning_rate=0.001) ### can be changed
los = tf.keras.losses.categorical_crossentropy  ### can be changed
  

model.compile(optimizer=opt, loss=los, metrics=['accuracy'])


In [ ]:
''' Training '''
model.fit(
    train_dataset, # both data and label cause we use ImageDataGenerator
    
    epochs=5, ### can be change 
    validation_data= validation_dataset,
    
    shuffle=True 
)


In [ ]:
''' Evaluating '''
print(model.history.history.keys())

In [ ]:
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper left')
plt.show()

In [ ]:
''' Predicting '''


# ชื่อไฟล์

 

*   740-821  = Orange
*   906-1005 = Clementine
*   911-1012 = Grapefruit *มี (2) ในชื่อที่ซ้ำ






In [ ]:
dir_path = '/content/drive/MyDrive/Images_of_Waste/test'
for i in os.listdir(dir_path):
    img = tf.keras.preprocessing.image.load_img(dir_path+'//'+i, target_size=(224,224))
    plt.imshow(img)
    plt.show()

    X= tf.keras.preprocessing.image.img_to_array(img)
    X= np.expand_dims(X,axis=0)
    image = np.vstack([X])
    val = model.predict(image)
    '''print(val)
    print(val[0][0])
    print(val[0][1])
    print(val[0][2])
    print(np.argmax(val)) '''

    if val[0][0] > val[0][1] and val[0][0] > val[0][2] :
        print('This is AluCan : '+i)
    
    elif val[0][1] > val[0][0] and val[0][1] > val[0][2] :
        print('This isGlass : '+i)

    elif val[0][2] > val[0][0] and val[0][2] > val[0][1] :
        print('This is PET : '+i)